In [1]:
!pip install pandas openpyxl

In [2]:
import pandas as pd

# Read the Excel file
file_path = 'dimension_play_log.xlsx'
df = pd.read_excel(file_path, engine='openpyxl')

df.columns = [col.lower().replace(' ', '_') for col in df.columns]


# Check the duplicate columns and rename them
cols = df.columns.tolist()
seen = {}
for idx, col in enumerate(cols):
    if col in seen:
        seen[col] += 1
        cols[idx] = f"{col}_{seen[col]}"
    else:
        seen[col] = 0

df.columns = cols

# get rid of the period
df.columns = [col.lower().replace('.', '_') for col in df.columns]
# Display the DataFrame to verify
df.head()


,timestamp,tasks,player_name,a,b,c,d,e,f,g,...,g_3,h_3,i_3,j_3,k_3,l_3,m_3,n_3,score_3,bonus_3
0,2023-08-24 13:29:14.587,"""QUANTITY-2-B"", ""BOTTOM-O"", ""TOUCH-K-K"", ""TOUC...",Andrew,Green,Black,Green,Orange,Orange,Orange,Green,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-08-24 13:43:33.112,"""NOTOUCH-G-G"", ""TOUCH-G-W"", ""TOUCH-G-K"", ""NOTO...",Andrew,Green,Black,Blue,Blue,White,Green,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-08-24 13:49:28.987,"""QUANTITY-1-K"", ""QUANTITY-1-B"", ""BOTTOM-K"", ""T...",Andrew,White,White,Black,White,Orange,Blue,Orange,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-08-24 13:54:45.847,"""QUANTITY-1-W"", ""QUANTITY-2-O"", ""NOTOUCH-G-K"",...",Andrew,Blue,Green,Green,Green,Blue,Black,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-08-24 14:02:42.970,"""BOTTOM-G"", ""BOTTOM-O"", ""NOTOUCH-K-K"", ""TOUCH-...",Andrew,Blue,White,Black,White,Black,White,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
print(df.columns.tolist())

['timestamp', 'tasks', 'player_name', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'score', 'bonus', 'player_name_1', 'a_1', 'b_1', 'c_1', 'd_1', 'e_1', 'f_1', 'g_1', 'h_1', 'i_1', 'j_1', 'k_1', 'l_1', 'm_1', 'n_1', 'score_1', 'bonus_1', 'player_name_2', 'a_2', 'b_2', 'c_2', 'd_2', 'e_2', 'f_2', 'g_2', 'h_2', 'i_2', 'j_2', 'k_2', 'l_2', 'm_2', 'n_2', 'score_2', 'bonus_2', 'player_name_3', 'a_3', 'b_3', 'c_3', 'd_3', 'e_3', 'f_3', 'g_3', 'h_3', 'i_3', 'j_3', 'k_3', 'l_3', 'm_3', 'n_3', 'score_3', 'bonus_3']


In [4]:
# Melt the data into a player, turn, task list per row
# List to store mini dataframes
dfs = []

# Split and transform for each player group
for i in range(4):  # Here 4 is the number of player groups you have (0 to 3)
    if i == 0:
        prefix = ''
    else:
        prefix = f'_{i}'
    
    # Filter out rows where player name is empty
    player_df = df[df[f'player_name{prefix}'] != '']

    # Select columns for this player
    cols = ['timestamp', 'tasks', f'player_name{prefix}', f'score{prefix}', f'bonus{prefix}'] + [f'{char}{prefix}' for char in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n']]
    
    subset = player_df[cols]
    
    # Rename columns
    new_cols = ['timestamp', 'tasks', 'name', 'score', 'bonus'] + list('abcdefghijklmn')
    subset.columns = new_cols
    
    dfs.append(subset)

# Combine all mini dataframes
result = pd.concat(dfs, axis=0).reset_index(drop=True)

# print(result)
# print(result.iloc[40]) #get one row


In [5]:
#drop all Nan rows by Name
result = result[result['name'].notna()]


In [6]:
print(type(result))

<class 'pandas.core.frame.DataFrame'>


In [7]:
def row_to_json(row):
    # For the dimensions
    dimensions = [chr(i) for i in range(ord('a'), ord('n')+1)]  # List of all dimensions from 'a' to 'n'
    dimension_data = {dim: str(row[dim]).upper() for dim in dimensions if not pd.isna(row[dim])}

   # Split tasks into a list and strip away quotes and spaces
    tasks_list = [task.strip().strip('"') for task in row['tasks'].split(',')]

    # Convert bonus to boolean and truncate score
    bonus_bool = bool(row['bonus'])
    score_int = int(row['score'])

    # Construct the final dictionary
    json_data = {
        'name': row['name'],
        'bonus': bonus_bool,
        'score': score_int,
        'tasks': tasks_list,
        'dimension': dimension_data
    }
    
    return json_data


# Apply the function to each row
json_list = result.apply(row_to_json, axis=1).tolist()



In [8]:
import os
import json

# ... [The rest of your code to generate the json_list]

# Ensure the 'test_cases' directory exists
if not os.path.exists('test_cases'):
    os.makedirs('test_cases')

# Save each JSON object to a separate file in 'test_cases' directory
for idx, json_obj in enumerate(json_list, start=1):
    filename = os.path.join('test_cases', f"TestCase{idx}.json")
    with open(filename, 'w') as file:
        json.dump(json_obj, file, indent=2)  # Pretty print with 2-space indentation
